# Lexical Emotion Classification of Different Climate Change Tweets

In [1]:
from emonal import Emo

In [2]:
lexipaths = {
    "VAD": "NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt",
    "emotion": "NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt",
    "affect intensity": "NRC-Affect-Intensity-Lexicon/NRC-AffectIntensity-Lexicon.txt"
}
big_folder = "NRC-Sentiment-Emotion-Lexicons"

In [3]:
with open("../data/tweets.txt","r",encoding="utf-8") as f:
    content = f.readlines()
with open("../data/labels.csv","r",encoding="utf-8") as f:
    labels = f.readlines()
pro = []
anti = []
unk = []
for i, c in enumerate(content):
    if labels[i].strip() == "1":
        pro.append(c)
    elif labels[i].strip() == "2":
        anti.append(c)
    elif labels[i].strip() == "3":
        unk.append(c)
print(len(pro))
print(len(anti))
print(len(unk))

954
337
708


In [4]:
emojudge = Emo(lexipaths,big_folder)

In [5]:
pro_vals = emojudge.classify(pro,"all")
anti_vals = emojudge.classify(anti,"all")
unk_vals = emojudge.classify(unk,"all")

In [7]:
print(pro_vals)
print(anti_vals)
print(unk_vals)

{'emotion': {'anger': 0.060719526732252144, 'anticipation': 0.06114657405749672, 'disgust': 0.04354069401789447, 'fear': 0.21411826950002988, 'joy': 0.04040893712155644, 'sadness': 0.057511794203204625, 'surprise': 0.03560950361056405, 'trust': 0.08743184763863458}, 'VAD': {'Valence': 0.5888856775894532, 'Arousal': 0.46384266156816295, 'Dominance': 0.552004782024265}, 'affect intensity': {'anger': 0.07847109722222224, 'fear': 0.19764638194444406, 'joy': 0.08425005555555558, 'sadness': 0.08212337499999998}}
{'emotion': {'anger': 0.08706941483060882, 'anticipation': 0.04215847171071049, 'disgust': 0.06921937929400612, 'fear': 0.21545153022764968, 'joy': 0.028615257048092868, 'sadness': 0.05437218668561949, 'surprise': 0.043797678275290214, 'trust': 0.07008054963278842}, 'VAD': {'Valence': 0.5707350169068081, 'Arousal': 0.4749765269970494, 'Dominance': 0.5477089017036034}, 'affect intensity': {'anger': 0.10759880095923256, 'fear': 0.1871582134292566, 'joy': 0.07092074340527578, 'sadness':

In [6]:
for k in pro_vals:
    for m in pro_vals[k]:
        if (diff := abs(pro_vals[k][m] - anti_vals[k][m])) > 0.02:
            print(f"{k}/{m}/pro-anti: {diff} ({pro_vals[k][m]} vs {anti_vals[k][m]})")
        if (diff := abs(pro_vals[k][m] - unk_vals[k][m])) > 0.02:
            print(f"{k}/{m}/pro-unk: {diff} ({pro_vals[k][m]} vs {unk_vals[k][m]})")
        if (diff := abs(anti_vals[k][m] - unk_vals[k][m])) > 0.02:
            print(f"{k}/{m}/anti-unk: {diff} ({anti_vals[k][m]} vs {unk_vals[k][m]})")

emotion/anger/pro-anti: 0.026349888098356672 (0.060719526732252144 vs 0.08706941483060882)
emotion/anger/anti-unk: 0.03329447151995122 (0.08706941483060882 vs 0.053774943310657596)
emotion/disgust/pro-anti: 0.025678685276111654 (0.04354069401789447 vs 0.06921937929400612)
emotion/disgust/anti-unk: 0.03478230446407417 (0.06921937929400612 vs 0.034437074829931955)
VAD/Valence/anti-unk: 0.021796481900651976 (0.5707350169068081 vs 0.59253149880746)
affect intensity/anger/pro-anti: 0.02912770373701032 (0.07847109722222224 vs 0.10759880095923256)
affect intensity/anger/anti-unk: 0.030092477082164018 (0.10759880095923256 vs 0.07750632387706854)
affect intensity/joy/anti-unk: 0.022350149029712413 (0.07092074340527578 vs 0.09327089243498819)
